In [ ]:
import pandas as pd  
import os
import csv
"""
reg_headers = pd.read_csv("/Users/jmitovich/Downloads/mail_ballots/reg_headers.csv", header = None, error_bad_lines = False)
headers = []
for val in reg_headers.values.tolist():
    headers.append(val[0])
"""

In [166]:
mail_ballots = {}
mail_ballots['2020_primary'] = pd.read_csv("/Users/jmitovich/Downloads/mail_ballots/SWMB.60320.txt",sep=",",encoding='cp1252') 
mail_ballots['2020_general'] = pd.read_csv("/Users/jmitovich/Downloads/mail_ballots/SWMB.11052020.txt", sep=",",encoding='cp1252',on_bad_lines='skip') 
mail_ballots['2021_primary'] = pd.read_csv("/Users/jmitovich/Downloads/mail_ballots/2021Primary_SWMailBallot_5.20.2021.txt", sep=",",encoding='cp1252',on_bad_lines='skip') 
mail_ballots['2021_general'] = pd.read_csv("/Users/jmitovich/Downloads/mail_ballots/2021General_SWMailBallot_11.04.txt", sep="|",encoding='cp1252',on_bad_lines='skip') 
mail_ballots['2022_primary'] = pd.read_csv("/Users/jmitovich/Downloads/mail_ballots/2022Primary_SWMailBallot_5.19.txt", sep="|",encoding='cp1252', on_bad_lines = 'skip') 
mail_ballots['2022_general'] = pd.read_csv("VR_SWMailBallot_External 20221025.txt", sep="|") 


/Users/jmitovich/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [174]:
from datetime import timedelta
from datetime import datetime
from decimal import ROUND_05UP
import numpy as np
import math

for e in mail_ballots:
    election = mail_ballots[e]
    election[['DateofBirth','AppIssueDate','AppReturnDate','BallotSentDate','BallotReturnedDate','ElectionDate']] = election[['DateofBirth','AppIssueDate','AppReturnDate','BallotSentDate','BallotReturnedDate','ElectionDate']].apply(pd.to_datetime,errors='coerce')
        
for e in mail_ballots:
    election = mail_ballots[e]
    election['AppIssue_dte'] = (election['ElectionDate'] - election['AppIssueDate']).dt.days
    election['AppReturn_dte'] = (election['ElectionDate'] - election['AppReturnDate']).dt.days
    election['BallotSent_dte'] = (election['ElectionDate'] - election['BallotSentDate']).dt.days
    election['BallotReturned_dte'] = (election['ElectionDate'] - election['BallotReturnedDate']).dt.days
    
cumul_requests = []
new_requests = []
null_series = pd.Series(index=list(range(0,151)),name='nulls')

for e in mail_ballots:
    election = mail_ballots[e]
    # get value of cumulative ballots requested through day t-151 of election
    cumul_151 = len(election[(election['AppReturn_dte']>150)])
    new_151 = len(election[(election['AppReturn_dte']>150)])

    # get cumulative ballots requested by days to election
    cumul_sr = election[(election['AppReturn_dte']<=150)&(election['AppReturn_dte']>0)].groupby('AppReturn_dte').size().fillna(0).sort_index(ascending=False).cumsum(skipna=True).ffill() + cumul_151
    #new_sr = election[(election['AppReturn_dte']<=150)&(election['AppReturn_dte']>0)].groupby('AppReturn_dte').size().fillna(0).sort_index(ascending=False).sum(skipna=True).ffill()

    # name it
    cumul_sr.name = e
    #new_sr.name = e
    
    # merge with null series series so it has something at each point between 0 and 150
    cumul_sr = pd.merge(null_series,cumul_sr,left_index=True,right_index=True,how='left')
    #new_sr = pd.merge(null_series,new_sr,left_index=True,right_index=True,how='left')

    # drop the null series, and ffill and fillna
    cumul_sr = cumul_sr.drop('nulls',axis=1).ffill().fillna(0)
    cumul_requests.append(cumul_sr)
    #new_sr = new_sr.drop('nulls',axis=1).ffill().fillna(0)
    #new_requests.append(new_sr)
    
cumul_df = pd.concat(cumul_requests,axis=1).sort_index(ascending=False).ffill()
#new_df = pd.concat(new_requests,axis=1).sort_index(ascending=False).ffill()

<ipython-input-174-70c74b124aca>:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  null_series = pd.Series(index=list(range(0,151)),name='nulls')


AttributeError: 'numpy.int64' object has no attribute 'ffill'

In [ ]:

from datetime import timedelta
from datetime import datetime
from decimal import ROUND_05UP
import numpy as np
import math

### Make DataFrame for later
cnt_req = pd.DataFrame()

for e in mail_ballots:
    
    ### Store DataFrame of the given election's mail ballot requests
    election = mail_ballots[e]
    
    ### Convert AppReturnDate to datetime object
    election['AppReturnDate_Date'] = [datetime.strptime(str(s), "%m/%d/%Y") for s in election['AppReturnDate']]
    election['AppReturnDate_Date'] = [datetime.strptime]

    ### Store the election dates so we can calculate the 150 days to election range
    election_dates = {'2020_primary': [datetime.strptime("2020-06-02", "%Y-%m-%d"), datetime.strptime("2020-06-02", "%Y-%m-%d") - timedelta(days=150)],
                  '2020_general': [datetime.strptime("2020-11-03", "%Y-%m-%d"), datetime.strptime("2020-11-03", "%Y-%m-%d") - timedelta(days=150)],
                  '2021_primary': [datetime.strptime("2021-05-18", "%Y-%m-%d"), datetime.strptime("2021-05-18", "%Y-%m-%d") - timedelta(days=150)],
                  '2021_general': [datetime.strptime("2021-11-02", "%Y-%m-%d"), datetime.strptime("2021-11-02", "%Y-%m-%d") - timedelta(days=150)],
                  '2022_primary': [datetime.strptime("2022-05-17", "%Y-%m-%d"), datetime.strptime("2022-05-17", "%Y-%m-%d") - timedelta(days=150)],
                  '2022_general': [datetime.strptime("2022-11-08", "%Y-%m-%d"), datetime.strptime("2022-11-08", "%Y-%m-%d") - timedelta(days=150)]}

    dates = sorted(election['AppReturnDate_Date'].unique())
    dates_ballots = []
    count_requests = []
    total_requests = 0
    end = election_dates[e][0]
    print("End:", end)
    start = end - timedelta(days=150)
    print("Start:", start)
    
    for date in (start + timedelta(n) for n in range(0, 150)):
        if(pd.isnull(date)):
            date = datetime.strptime("1900-01-01", "%Y-%m-%d")
        else:
            date = datetime.strptime(datetime.strftime(date, "%Y-%m-%d"), "%Y-%m-%d")
        if np.isnat(date_time):
            print("Removed:", date)
        year = datetime.strptime(datetime.strftime(date, "%Y-%m-%d"), "%Y-%m-%d").year
        if year >= 1901 and date < start and date == date:
            dates_ballots.append(date)
        
    print(dates_ballots)    
    dates_final = []
    total_req = []
    diff_days = []
    
    ### Count ballot requests before 150 days
    for date in sorted(dates_ballots):
        if pd.isnull(date):
            print("Null:", date)
            dates.remove(date)
        return_date = datetime.strftime(date, "%m/%d/%Y")  
        print(return_date)                        
        try:
            total_requests = total_requests + election['AppReturnDate_Date'].value_counts()[return_date]
        except KeyError:
            total_requests = total_requests
            pass
        
    ### Count ballot requests after 150 days and add the cumulative number per date to the data frame
    print(total_requests)
    for date_range in (start + timedelta(n) for n in range(0, 150)):
        
        return_date = datetime.strftime(date_range, "%m/%d/%Y")                          
        try:
            ### Count new total_requests at that time if that date has a record
            total_requests = total_requests + election['AppReturnDate_Date'].value_counts()[return_date]
        except KeyError:
            ### Otherwise just keep it what it was from the last recorded date
            total_requests = total_requests
            pass
    
        ### Append all the new values:
        
        ### Date at the time
        dates_final.append(date_range)
        ### Cumulative requests at that date
        total_req.append(total_requests)
        ### Days before the election at that date
        diff_days.append((end - date_range).days)
        
    cnt_req['date_{}'.format(e)] = dates_final
    cnt_req['total_req_{}'.format(e)] = total_req
    cnt_req['diff_days_{}'.format(e)] = diff_days


    
    #pd.DataFrame(count_requests, columns = ["Date", "Total", "diff.days"])

    

In [170]:
cumul_df.to_csv("/Users/jmitovich/Downloads/mail_ballots/requests_2020_2022.csv")
new_df.to_csv("/Users/jmitovich/Downloads/mail_ballots/new_requests_2020_2022.csv")

In [ ]:
"""
directory_in_str = "/Users/jmitovich/Downloads/mail_ballots/Statewide_20221017"
directory = os.fsencode(directory_in_str)
county_files = []

for file in os.listdir(directory):
     filename = os.fsdecode(file)
     if filename.endswith(".txt") and "FVE" in filename: 
         new_file = directory_in_str + "/" + filename
         county_files.append(new_file)
     else:
         continue
     
dfs = []
for filename in county_files:
    county = pd.read_csv(filename, sep = "\t", header = 0, error_bad_lines = False, names = headers)
    dfs.append(county)
    
big_frame = pd.concat(dfs, ignore_index=True)
"""

In [ ]:
#big_frame.loc[:,'id_number']
#big_frame.rename(columns={'id_number': 'IDNumber'}, inplace = True)

In [ ]:
#big_frame.head()

#results = pd.merge(ballots, big_frame, on=["IDNumber"])

In [ ]:
"/Users/jmitovich/Downloads/mail_ballots/all_voters.csv"#results.to_csv()

In [ ]:
"""
from datetime import datetime
import numpy as np
import math
ballots['AppReturnDate'] = pd.to_datetime(ballots['AppReturnDate'], format = "%m/%d/%Y")
dates = sorted(pd.to_datetime(ballots['AppReturnDate'].unique())) 

dates_2022 = []
for date in dates:
    if(pd.isnull(date)):
        date = pd.to_datetime("1900-01-01", format = "%Y-%m-%d")
    else:
        date_time = np.datetime64(date)
    if np.isnat(date_time):
        print("Removed:", date)
    year = datetime.strptime(datetime.strftime(date, "%Y-%m-%d"), "%Y-%m-%d").year
    if year >= 1901 and date == date:
        dates_2022.append(date)
        
count_requests = []
total_requests = 0
total_requests_r = 0
total_requests_d = 0
total_requests_philly = 0
total_requests_montco = 0
total_requests_delco = 0
total_requests_bucksco = 0
total_requests_chesterco = 0

counties = ['Philadelphia', 'Montgomery','Delaware','Bucks','Chester']

for date in sorted(dates_2022):
    if pd.isnull(date):
        print("Null:", date)
        dates.remove(date)
    total_requests = total_requests + ballots['AppReturnDate'].value_counts()[date]
    try:
        total_requests_d = total_requests_d + ballots[ballots['Party'] == 'D']['AppReturnDate'].value_counts()[date]
    except KeyError:
        total_requests_d = total_requests_d
        pass
    try:
        total_requests_r = total_requests_r + ballots[ballots['Party'] == 'R']['AppReturnDate'].value_counts()[date]
    except KeyError:
        total_requests_r = total_requests_r
        pass
    try:
        total_requests_philly = total_requests_philly + ballots[ballots['CountyName'] == 'PHILADELPHIA']['AppReturnDate'].value_counts()[date]
    except KeyError:
        total_requests_philly = total_requests_philly
        pass
    try:
        total_requests_montco = total_requests_montco + ballots[ballots['CountyName'] == 'MONTGOMERY']['AppReturnDate'].value_counts()[date]
    except KeyError:
        total_requests_montco = total_requests_montco
        pass
    try:
        total_requests_delco = total_requests_delco + ballots[ballots['CountyName'] == 'DELAWARE']['AppReturnDate'].value_counts()[date]
    except KeyError:
        total_requests_delco = total_requests_delco
        pass
    try:
        total_requests_chesterco = total_requests_chesterco + ballots[ballots['CountyName'] == 'CHESTER']['AppReturnDate'].value_counts()[date]
    except KeyError:
        total_requests_chesterco = total_requests_chesterco
        pass
    
    try:
        total_requests_bucksco = total_requests_bucksco + ballots[ballots['CountyName'] == 'BUCKS']['AppReturnDate'].value_counts()[date]
    except KeyError:
        total_requests_bucksco = total_requests_bucksco
        pass
    count_requests.append([date, total_requests, total_requests_d, total_requests_r, total_requests_d / total_requests, total_requests_r / total_requests,
                           total_requests_philly,
                           total_requests_montco,
                           total_requests_delco,
                           total_requests_chesterco,
                           total_requests_bucksco,
                           (total_requests_philly +
                           total_requests_montco+
                           total_requests_delco+
                           total_requests_chesterco+
                           total_requests_bucksco) / total_requests])
    
cnt_req = pd.DataFrame(count_requests, columns = ["Date", "Total", "Democrat", "Republican", "Democrat Pct", "Republican Pct", 
                                                  "Philadelphia",
                                                  "Montgomery",
                                                  "Delaware",
                                                  "Chester",
                                                  "Bucks",
                                                  "Philly + Suburbs Pct"])

"""

In [ ]:
"""  
pd.DataFrame.from_dict(count_requests, orient='index').to_csv("/Users/jmitovich/Downloads/mail_ballots/requests_2022_general.csv")
"""

In [ ]:
"""
dates = results['AppReturnDate'].tolist()

for date in dates:
    date = datetime.strptime(datetime.strftime(date, "%Y-%m-%d"), "%Y-%m-%d")
print(dates)
"""

In [ ]:
""" 
from datetime import timedelta
from datetime import datetime
from decimal import ROUND_05UP
import numpy as np
import math

cnt_req = pd.DataFrame()

for e in mail_ballots:
    election = mail_ballots[e]
    election_dates = {'2020_primary': [datetime.strptime("2020-06-02", "%Y-%m-%d"), datetime.strptime("2020-06-02", "%Y-%m-%d") - timedelta(days=150)],
                  '2020_general': [datetime.strptime("2020-11-03", "%Y-%m-%d"), datetime.strptime("2020-11-03", "%Y-%m-%d") - timedelta(days=150)],
                  '2021_primary': [datetime.strptime("2021-05-18", "%Y-%m-%d"), datetime.strptime("2021-05-18", "%Y-%m-%d") - timedelta(days=150)],
                  '2021_general': [datetime.strptime("2021-11-02", "%Y-%m-%d"), datetime.strptime("2021-11-02", "%Y-%m-%d") - timedelta(days=150)],
                  '2022_primary': [datetime.strptime("2022-05-17", "%Y-%m-%d"), datetime.strptime("2022-05-17", "%Y-%m-%d") - timedelta(days=150)],
                  '2022_general': [datetime.strptime("2022-11-08", "%Y-%m-%d"), datetime.strptime("2022-11-08", "%Y-%m-%d") - timedelta(days=150)]}

    dates_ballots = []
    for date in dates:
        if(pd.isnull(date)):
            date = pd.to_datetime("1900-01-01", format = "%Y-%m-%d")
        else:
            date_time = np.datetime64(date)
        if np.isnat(date_time):
            print("Removed:", date)
        year = datetime.strptime(datetime.strftime(date, "%Y-%m-%d"), "%Y-%m-%d").year
        if year >= 1901 and date == date:
            dates_ballots.append(date)
            
    count_requests = []
    total_requests = 0
    #total_requests_r = 0
    #total_requests_d = 0
    #total_requests_philly = 0
    #total_requests_montco = 0
    #total_requests_delco = 0
    #total_requests_bucksco = 0
    #total_requests_chesterco = 0

    #counties = ['Philadelphia', 'Montgomery','Delaware','Bucks','Chester']

    end = election_dates[e][0]
    print(end)
    
    dates_final = []
    total_req = []
    diff_days = []

    #cnt_req['index_{}'.format(e)] = range(1, 150)
    #cnt_req['date_{}'.format(e)] = range(1, 150)
    #cnt_req['total_req_{}'.format(e)] = range(1, 150)
    #cnt_req['diff_days_{}'.format(e)] = range(1, 150)
    
    #for i in range(1, 150):
        #cnt_req[i]['index_{}'.format(e)] = 0
        #cnt_req[i]['date_{}'.format(e)] = 0
        #cnt_req[i]['total_req_{}'.format(e)] = 0
        #cnt_req[i]['diff_days_{}'.format(e)] = 0
    
    for date in sorted(dates_ballots):
        if pd.isnull(date):
            print("Null:", date)
            dates.remove(date)
            
        try:
            total_requests = total_requests + election['AppReturnDate'].value_counts()[date]
        except KeyError:
            total_requests = total_requests
            pass
        
        
        start = datetime.strptime(datetime.strftime(date, "%Y-%m-%d"), "%Y-%m-%d")
        diff = end - start
        
        days_range = range(0, 150)
        #for day in days_range:
           # if day == diff.days:
        dates_final.append(date)
        total_req.append(total_requests)
        diff_days.append(diff.days)
        count_requests.append([date, total_requests, diff.days])
        
    cnt_req['index_{}'.format(e)] = days_range
    cnt_req['date_{}'.format(e)] = dates_final
    cnt_req['total_req_{}'.format(e)] = total_req
    cnt_req['diff_days_{}'.format(e)] = diff_days


    
    #pd.DataFrame(count_requests, columns = ["Date", "Total", "diff.days"])

    """

In [ ]:
"""
requests_list = []

for election in mail_ballots:
    
    election['AppReturnDate'] = pd.to_datetime(election['AppReturnDate'], format = "%m/%d/%Y")
    dates = sorted(pd.to_datetime(election['AppReturnDate'].unique())) 

    dates_2022 = []
    for date in dates:
        date = datetime.strptime(datetime.strftime(date, "%Y-%m-%d"), "%Y-%m-%d")
        if(pd.isnull(date)):
            date = pd.to_datetime("1900-01-01", format = "%Y-%m-%d")
        else:
            date_time = np.datetime64(date)
        if np.isnat(date_time):
            print("Removed:", date)
        year = datetime.strptime(datetime.strftime(date, "%Y-%m-%d"), "%Y-%m-%d").year
        if year >= 1901 and date == date:
            dates_2022.append(date)
            
    count_requests = {}
    total_requests = 0
    for date in sorted(dates_2022):
        if pd.isnull(date):
            print("Null:", date)
            dates.remove(date)
        total_requests = total_requests + election['AppReturnDate'].value_counts()[date]
        count_requests[date] = total_requests 
    requests_list.append(count_requests)
"""